# Notebook 04 — Budget-Constrained Transfer Optimisation (Final)
### Risk-adjusted scouting decision support (MILP)

This notebook turns the scouting outputs from **Notebook 02 (Talent Score)** and **Notebook 03 (Risk-Adjusted Value)** into an **operations research** decision layer.

We solve a **budget-constrained selection problem** for *position-specific reinforcements* (wingers / attacking midfielders / attackers), using:

- **Binary decision variables** (sign / do not sign)
- **Budget constraint** (real market value, €)
- **K signings constraint** (choose exactly K players)
- **Risk-adjusted objective** with a tunable **λ** trade-off parameter
- **Scenario simulation** (budget and risk appetite)
- Exportable outputs to `/reports`

> Note: because our scouting universe is **already filtered** to attacking profiles upstream, we do **not** impose full-squad positional structure constraints (GK/DF/etc.).


## 1) Imports & Environment Configuration

In [ ]:
from __future__ import annotations

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from dataclasses import dataclass

import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SciPy MILP (HiGHS)
from scipy.optimize import milp, LinearConstraint, Bounds

np.random.seed(42)

# --- Paths ---
NOTEBOOKS_DIR = Path.cwd().resolve()
REPO_ROOT = NOTEBOOKS_DIR.parent
DB_PATH = REPO_ROOT / "db" / "scouting.duckdb"

# Optimisation universe enriched with Transfermarkt market values + positions
IN_TABLE = "risk_adjusted_universe_v2"

print("CWD:", NOTEBOOKS_DIR)
print("Repo root:", REPO_ROOT)
print("DB path:", DB_PATH)
print("DB exists:", DB_PATH.exists())


## 2) Load Optimisation Universe (DuckDB)

In [ ]:
def load_universe(db_path: Path, table: str) -> pd.DataFrame:
    if not db_path.exists():
        raise FileNotFoundError(f"DuckDB not found at {db_path.resolve()}")
    with duckdb.connect(str(db_path)) as con:
        return con.execute(f"SELECT * FROM {table}").df()

df_raw = load_universe(DB_PATH, IN_TABLE)

print("Rows:", df_raw.shape[0], "Cols:", df_raw.shape[1])
df_raw.head(3)


## 3) Data Preparation: Cost (Market Value) and Objective Construction

### Cost
We use **Transfermarkt market value** (from `dim_player.market_value_in_eur`) as the optimisation cost.

- Units are converted to **€ millions** to keep budgets readable.

### Objective
We optimise a **risk-adjusted talent** score:

\[
\max \sum_i x_i (Talent_i - \lambda \cdot Risk_i)
\]

- `Talent_i` is the standardised `talent_score`.
- `Risk_i` is `risk_score` (note: in our current scoring, *more negative* values indicate safer profiles).
- `λ` controls risk aversion (higher λ → more weight on risk).


In [ ]:
df = df_raw.copy()

# Cost in €m
df["cost"] = pd.to_numeric(df["market_value_in_eur"], errors="coerce") / 1_000_000
df = df.dropna(subset=["cost"]).copy()

# Core columns checks
required_cols = ["player_id","player_name","age","position","talent_score","risk_score","cost"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

print("Universe size:", len(df))
print("Average market value (€m):", round(df["cost"].mean(), 2))
print("Cost stats (€m):")
df["cost"].describe()[["min","mean","max"]]


## 4) Formal Optimisation Problem

We solve a **Mixed-Integer Linear Program (MILP)**.

**Decision variable**
- \(x_i \in \{0,1\}\): select player \(i\) (sign) or not.

**Objective**
- Maximise total risk-adjusted talent:
\[
\max \sum_i x_i (Talent_i - \lambda \cdot Risk_i)
\]

**Constraints**
1. **K signings**
\[
\sum_i x_i = K
\]

2. **Budget**
\[
\sum_i x_i \cdot Cost_i \le B
\]

Optional constraints (not used here, but easy to add): age cap, homegrown quotas, league quotas, etc.


## 5) MILP Solver Implementation (SciPy / HiGHS)

In [ ]:
@dataclass
class SolveResult:
    success: bool
    message: str
    lambda_: float
    k: int
    budget: float
    objective_value: float
    total_cost: float
    total_talent: float
    total_risk: float
    selected: pd.DataFrame


def solve_signings(
    dfin: pd.DataFrame,
    *,
    lambda_: float,
    k: int,
    budget: float,
) -> SolveResult:
    d = dfin.copy()

    # objective_score = talent - lambda * risk
    d["objective_score"] = d["talent_score"].astype(float) - lambda_ * d["risk_score"].astype(float)

    n = len(d)
    c = -d["objective_score"].to_numpy(dtype=float)  # milp minimizes
    integrality = np.ones(n, dtype=int)
    bounds = Bounds(lb=np.zeros(n), ub=np.ones(n))

    A = []
    lb = []
    ub = []

    # K signings equality
    A.append(np.ones(n))
    lb.append(k)
    ub.append(k)

    # Budget inequality
    A.append(d["cost"].to_numpy(dtype=float))
    lb.append(-np.inf)
    ub.append(budget)

    constraints = LinearConstraint(np.vstack(A), lb=np.array(lb, float), ub=np.array(ub, float))

    res = milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)

    if not res.success:
        return SolveResult(
            success=False,
            message=res.message,
            lambda_=lambda_,
            k=k,
            budget=budget,
            objective_value=float("nan"),
            total_cost=float("nan"),
            total_talent=float("nan"),
            total_risk=float("nan"),
            selected=pd.DataFrame(),
        )

    x = np.round(res.x).astype(int)
    sel = d.loc[x == 1, ["player_id","player_name","age","position","cost","talent_score","risk_score","objective_score"]].copy()
    sel = sel.sort_values("objective_score", ascending=False)

    return SolveResult(
        success=True,
        message=res.message,
        lambda_=lambda_,
        k=k,
        budget=budget,
        objective_value=float(sel["objective_score"].sum()),
        total_cost=float(sel["cost"].sum()),
        total_talent=float(sel["talent_score"].sum()),
        total_risk=float(sel["risk_score"].sum()),
        selected=sel,
    )


## 6) Baseline Optimisation Run (Example)

We run a baseline configuration to produce a concrete, interpretable recommendation.

- **K = 3** signings (position-specific reinforcements)
- **Budget = €180m**
- **λ = 0.5**


In [ ]:
LAMBDA = 0.5
K_SIGNINGS = 3
BUDGET = 180.0  # €m

# Feasibility check: K cheapest must fit in the budget
min_feasible_cost = float(df["cost"].nsmallest(K_SIGNINGS).sum())
print(f"Cheapest feasible total for K={K_SIGNINGS}: {min_feasible_cost:.1f} €m")

out = solve_signings(df, lambda_=LAMBDA, k=K_SIGNINGS, budget=BUDGET)

print("Solver success:", out.success)
print("Solver message:", out.message)

if out.success:
    print("Objective value:", round(out.objective_value, 3))
    print("Total cost (€m):", round(out.total_cost, 1))
    print("Avg cost (€m):", round(out.total_cost / K_SIGNINGS, 1))
    print("Total talent:", round(out.total_talent, 3))
    print("Total risk:", round(out.total_risk, 3))

out.selected


## 7) Multi-objective Frontier: λ Sweep

We approximate a risk–talent trade-off frontier by sweeping λ over a grid and resolving the MILP.

Interpretation:
- Higher λ increases the weight on risk (more risk-averse).
- Changes in the optimal solution often occur in **regime shifts** (discrete swaps), which is expected in integer optimisation.


In [ ]:
lambdas = np.linspace(0.0, 2.0, 21)

rows = []
for lam in lambdas:
    res = solve_signings(df, lambda_=float(lam), k=K_SIGNINGS, budget=BUDGET)
    rows.append({
        "lambda": float(lam),
        "success": bool(res.success),
        "objective": res.objective_value,
        "total_cost": res.total_cost,
        "total_talent": res.total_talent,
        "total_risk": res.total_risk,
        "avg_talent": res.total_talent / K_SIGNINGS if res.success else np.nan,
        "avg_risk": res.total_risk / K_SIGNINGS if res.success else np.nan,
    })

frontier = pd.DataFrame(rows)
frontier


In [ ]:
plt.figure()
plt.plot(frontier["lambda"], frontier["objective"], marker="o")
plt.xlabel("lambda (risk penalty)")
plt.ylabel("Optimal objective (sum)")
plt.title("Risk-Adjusted Objective vs Lambda (K=3, fixed budget)")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(frontier["total_risk"], frontier["total_talent"], marker="o")
plt.xlabel("Total risk (sum)")
plt.ylabel("Total talent (sum)")
plt.title("Talent–Risk Trade-off Frontier (lambda sweep)")
plt.grid(True)
plt.show()


## 8) Scenario Simulation (Budget × Risk Appetite)

We run a small set of club-like scenarios to stress-test decisions under different constraints.

- **Tight budget**: less money available for the same K signings
- **Baseline**: the reference case
- **Risk-averse**: higher λ with the same budget


In [ ]:
SCENARIOS = [
    {"name": "Tight budget", "K": 3, "budget": 140.0, "lambda": 0.5},
    {"name": "Baseline",     "K": 3, "budget": 180.0, "lambda": 0.5},
    {"name": "Risk-averse",  "K": 3, "budget": 180.0, "lambda": 1.5},
]

scenario_rows = []
scenario_squads = {}

for sc in SCENARIOS:
    res = solve_signings(df, lambda_=sc["lambda"], k=sc["K"], budget=sc["budget"])
    scenario_rows.append({
        "scenario": sc["name"],
        "success": res.success,
        "K": sc["K"],
        "budget": sc["budget"],
        "lambda": sc["lambda"],
        "total_cost": res.total_cost,
        "total_talent": res.total_talent,
        "total_risk": res.total_risk,
        "objective": res.objective_value,
    })
    if res.success:
        scenario_squads[sc["name"]] = res.selected

scenario_report = pd.DataFrame(scenario_rows)
scenario_report


In [ ]:
plt.figure()
plt.plot(scenario_report["budget"], scenario_report["objective"], marker="o")
plt.xlabel("Budget (€m)")
plt.ylabel("Objective value")
plt.title("Objective vs Budget (Scenarios)")
plt.grid(True)
plt.show()


## 9) Export Outputs (CSV)

Exports are written to the repository `reports/` folder:
- `scenario_report_notebook04.csv`
- `squad_<scenario>.csv` for each feasible scenario


In [ ]:
REPORTS_DIR = REPO_ROOT / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

scenario_path = REPORTS_DIR / "scenario_report_notebook04.csv"
scenario_report.to_csv(scenario_path, index=False)
print("Saved:", scenario_path)

def safe_name(s: str) -> str:
    return "".join(ch if ch.isalnum() else "_" for ch in s).lower()

for name, squad in scenario_squads.items():
    path = REPORTS_DIR / f"squad_{safe_name(name)}.csv"
    squad.to_csv(path, index=False)
    print("Saved:", path)


## 10) Conclusions

### What this notebook demonstrates

- A **formal MILP formulation** for transfer decision-making (binary selection under budget).
- A practical **risk-adjusted objective** linking scouting scores (talent) with availability/instability risk.
- A **λ frontier** showing how optimal recommendations shift as the club becomes more risk-averse.
- **Scenario simulation** for realistic constraints (tight budget vs baseline vs risk-averse).

### Key takeaways

- Under a **fixed K-signings policy**, budget controls the attainable talent ceiling.
- Increasing λ (risk aversion) can change optimal picks in **discrete regimes**, consistent with integer optimisation.
- Using **real market values** makes the optimisation directly interpretable as a transfer strategy tool.

### Next upgrade (optional)

- Add **age caps**, **homegrown constraints**, and **league/club eligibility** filters.
- Replace market value with a **total acquisition cost model** (fee + wage + contract length) if available.
